# Approche ML 2.1 : XGboost & Optuna
Nous souhaitons dans ce notebook comprendre comment se comporte l'algorithme Xgboost sur le problème de classification des DCO. Sachant tout de même que lalimite dumodèle XGboost et son temps d'execution lorsque le nombre de classes est important (ici 449).

Les methodes traditionnelles pour optimiser les hyperparamètres  sont :
- Grid Search (mise en oeuvre pour le SVM dans l'approche 2.)
- Randomized Search : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

Ces méthodes sont trop longues pour notre problème car un entrainement nécessite environ 2h de calcul. Afin de sélectionner de manière pertinente les hyperparamèytres nous allons utliser l'optimisation bayesienne. Détailler dans cette article de manière intéressante : https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

En python, il existe différenteframeword pour mettre en place cette optimisation bayesienne :
- Optuna
- Hyperopt

Après la lecture de cet article (https://towardsdatascience.com/optuna-vs-hyperopt-which-hyperparameter-optimization-library-should-you-choose-ed8564618151) et les conseils de Boris nous avons choisis optuna.

Pour écrire le code, nous nous sommes inspirés de ces examples : 
- https://machinelearningapplied.com/hyperparameter-search-with-optuna-part-2-xgboost-classification-and-ensembling/
- https://machinelearningapplied.com/hyperparameter-search-and-pruning-with-optuna-part-4-xgboost-classification-and-ensembling/
- https://www.kaggle.com/kst6690/make-your-xgboost-model-awesome-with-optuna
- https://www.kaggle.com/snakayama/xgboost-using-optuna
- https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407

et utilisé la doc: 
https://optuna.readthedocs.io/en/latest/tutorial/rdb.html


In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd



import clean_text

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS


import xgboost as xgb

import optuna
from optuna import Trial

In [2]:
%%time
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv_complet.csv")#delimiter=';',encoding='ISO-8859-1')
id_to_dco = pd.read_csv("data/ref_MRV/referentiel_dispositif.csv",delimiter=';',encoding='ISO-8859-1')

#Charegement des colonnes utiles
df = df_declaration_mrv[['DESCRIPTION_INCIDENT','TYPE_VIGILANCE','LIBELLE_COMMERCIAL',
                         'REFERENCE_COMMERCIALE','ETAT_PATIENT','FABRICANT','DCO_ID','CLASSIFICATION']]
# On complète les NaN avec du vide
df['ETAT_PATIENT'] = df['ETAT_PATIENT'].fillna("")
df['DESCRIPTION_INCIDENT'] = df['DESCRIPTION_INCIDENT'].fillna("")
df['LIBELLE_COMMERCIAL'] = df['LIBELLE_COMMERCIAL'].fillna("")
df['FABRICANT'] = df['FABRICANT'].fillna("")
df["REFERENCE_COMMERCIALE"] = df['REFERENCE_COMMERCIALE'].fillna("")
df['TYPE_VIGILANCE'] = df['TYPE_VIGILANCE'].fillna("")
df['CLASSIFICATION'] = df['CLASSIFICATION'].fillna('')


# On ajoute des collones pertinentes
df['des_lib'] = df['LIBELLE_COMMERCIAL']+ ' ' + df['DESCRIPTION_INCIDENT']
df['fab_lib'] = df['LIBELLE_COMMERCIAL']+ ' ' + df['FABRICANT']
df['com'] = df['LIBELLE_COMMERCIAL']+ ' ' + df['REFERENCE_COMMERCIALE']
df['Text'] = df['LIBELLE_COMMERCIAL']+ ' ' + df['FABRICANT'] + "" + df['DESCRIPTION_INCIDENT']

# On nettoie les données :
for col in  ['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','ETAT_PATIENT','Text',"des_lib","fab_lib"] :
    df[col] = df[col].map(lambda x: clean_text.preprocess_text(x))

n = 15
# On filtre pour a voir plus de n observations par classse
df_n = df.groupby("DCO_ID").filter(lambda x: len(x) > n)

# On encode les labels
le = LabelEncoder()
df_n.DCO_ID = le.fit_transform(df_n.DCO_ID.values)
#On encode le type de vigilance
df_n.TYPE_VIGILANCE = le.fit_transform(df_n.TYPE_VIGILANCE.values)
#On encode la classifcation 
df_n.CLASSIFICATION = le.fit_transform(df_n.CLASSIFICATION.values)

# On selection les variables de test en faisant attention aux doublons
train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df_n, groups=df_n['DESCRIPTION_INCIDENT']))
df_train, df_test = df_n.iloc[train_index], df_n.iloc[test_index]



# On transforme les variables
preprocess = ColumnTransformer(
    [   
     ('libelle_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'LIBELLE_COMMERCIAL'),
     
     ('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=5,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
    ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'FABRICANT')],
    
    remainder='passthrough')

X = df_train[['DESCRIPTION_INCIDENT','FABRICANT','LIBELLE_COMMERCIAL']]  
X_prep = preprocess.fit_transform(X)
X_test = df_test[['DESCRIPTION_INCIDENT','FABRICANT','LIBELLE_COMMERCIAL']]  
X_test_prep = preprocess.transform(X_test)

y_true = df_test.DCO_ID 
y = df_train.DCO_ID

CPU times: user 48.1 s, sys: 532 ms, total: 48.7 s
Wall time: 49 s


In [3]:
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial:Trial):
    data,target = X_prep, y
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.1)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)
    
    # Liste des paramètres à optimiser
    param = {
        #"silent": 1,
        "verbosity" :1,
        "objective":'multi:softmax',
        "eval_metric":'mlogloss',
        "num_class":len(y.unique()),
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),#btree and dart use tree based models while gblinear uses linear functions
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0), #L2 regularization term on weights. Increasing this value will make model more conservative.
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0), #L1 regularization term on weights. Increasing this value will make model more conservative.
        "nthread":-1
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 2, 9) #Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.
        param["eta"] = trial.suggest_loguniform("eta", 1e-5, 1.0)#alias: learning_rate
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)#lias: min_split_loss
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])#Controls a way new nodes are added to the tree
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])#Type of sampling algorithm
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"]) #Type of normalization algorithm
        param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0) #Dropout rate
        param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)#Probability of skipping the dropout procedure during a boosting iteration
    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial,'eval-mlogloss')
    
    bst = xgb.train(param, dtrain,20,evals=[(dvalid, "eval")],callbacks=[pruning_callback],early_stopping_rounds=15)
    preds = bst.predict(dvalid)
    #pred_labels = np.rint(preds)
    f1_weighted = f1_score(valid_y, pred_labels, average='weighted')
    pprint()
    bst.save_model('last_xgb.model')
            
    return f1_weighted
   

In [4]:
%%time
studyName = 'test_study'
maximum_time = 3*60*60#second
number_of_random_points = 25
#optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = studyName,  direction="maximize")
study.optimize(objective, n_trials=15, timeout=maximum_time)# On créer 15 points
#Alternative

#Suvegarde du resultat
df = study.trials_dataframe()
df.to_json(studyName+'.json')
print(study.best_trial)

[0]	eval-mlogloss:4.48689
Will train until eval-mlogloss hasn't improved in 30 rounds.
[1]	eval-mlogloss:36.76839
[2]	eval-mlogloss:27.88941
[3]	eval-mlogloss:35.42197
[4]	eval-mlogloss:35.66395
[5]	eval-mlogloss:35.80046
[6]	eval-mlogloss:27.88941
[7]	eval-mlogloss:36.25961
[8]	eval-mlogloss:36.32786
[9]	eval-mlogloss:36.25340
[10]	eval-mlogloss:27.88941
[11]	eval-mlogloss:36.24720
[12]	eval-mlogloss:36.30925
[13]	eval-mlogloss:36.29684
[14]	eval-mlogloss:27.88941
[15]	eval-mlogloss:36.24720
[16]	eval-mlogloss:36.37750
[17]	eval-mlogloss:36.45195
[18]	eval-mlogloss:27.88941
[19]	eval-mlogloss:36.43334
[0]	eval-mlogloss:6.10659
Will train until eval-mlogloss hasn't improved in 30 rounds.
[1]	eval-mlogloss:6.10635
[2]	eval-mlogloss:6.10588
[3]	eval-mlogloss:6.10561
[4]	eval-mlogloss:6.10526
[5]	eval-mlogloss:6.10475
[6]	eval-mlogloss:6.10458
[7]	eval-mlogloss:6.10406
[8]	eval-mlogloss:6.10373
[9]	eval-mlogloss:6.10322
[10]	eval-mlogloss:6.10297
[11]	eval-mlogloss:6.10273
[12]	eval-mlogl

## Resultat de 14h de run en optimisant la loss

In [17]:
df = pd.read_json('14_h_study.json')

In [24]:
df = df.sort_values('value')

In [26]:
df.loc[4].to_dict()

{'number': 4,
 'value': 0.703457,
 'datetime_start': 1590765186400,
 'datetime_complete': 1590770377614,
 'duration': 5191213,
 'params_alpha': 0.0131112178,
 'params_booster': 'dart',
 'params_eta': 0.1366919939,
 'params_gamma': 0.00010909920000000001,
 'params_grow_policy': 'depthwise',
 'params_lambda': 2.05748e-05,
 'params_max_depth': 7.0,
 'params_normalize_type': 'tree',
 'params_rate_drop': 4.187e-06,
 'params_sample_type': 'weighted',
 'params_skip_drop': 8.24e-08,
 'state': 'COMPLETE'}

In [28]:
%%time

data,target = X_prep, y
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.1)
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(valid_x, label=valid_y)

xgb_params = {'alpha': 0.013,
              'booster': 'dart',
              'eta': 0.13,
              'gamma': 0.0010,
              'grow_policy': 'depthwise',
              'lambda':2.05748e-05,
              'max_depth': 7,
              'normalize_type': 'tree',
              'rate_drop': 4.187e-06,
              'sample_type': 'weighted',
              'skip_drop': 8.24e-08,
              'objective': 'multi:softmax', 
              'eval_metric': 'mlogloss', 
              'seed': 23,
              'num_class':len(y.unique())
            }

bst = xgb.train(xgb_params, dtrain,20,evals=[(dvalid, "eval")])

[0]	eval-mlogloss:2.87413
[1]	eval-mlogloss:2.24748
[2]	eval-mlogloss:1.88546
[3]	eval-mlogloss:1.63839
[4]	eval-mlogloss:1.48865
[5]	eval-mlogloss:1.30299
[6]	eval-mlogloss:1.19614
[7]	eval-mlogloss:1.11411
[8]	eval-mlogloss:1.04723
[9]	eval-mlogloss:0.99068
[10]	eval-mlogloss:0.93931
[11]	eval-mlogloss:0.89266
[12]	eval-mlogloss:0.85504
[13]	eval-mlogloss:0.81862
[14]	eval-mlogloss:0.78683
[15]	eval-mlogloss:0.75795
[16]	eval-mlogloss:0.73166
[17]	eval-mlogloss:0.70775
[18]	eval-mlogloss:0.68668
[19]	eval-mlogloss:0.66736
CPU times: user 2h 55min 14s, sys: 39.1 s, total: 2h 55min 53s
Wall time: 12min 59s


In [29]:
y_pred = bst.predict( xgb.DMatrix(X_test_prep))

In [30]:
print('précision:',accuracy_score(y_pred,y_true))
print('présision pondéré: ', balanced_accuracy_score(y_pred,y_true))
print('f1_weighted : ',f1_score(y_pred,y_true,average='weighted'))

précision: 0.7156460693681796
présision pondéré:  0.6228305996165773
f1_weighted :  0.7370516663765648


### Commentaire : 
Pour interpréter la Loss dans le cas ou nous avons 449 classes équilibrées, il faut savoir que -log (1/449) = 6.10, représente le seuil de "connaissance".
Nos classes étant fortment déséquilibré, si l'onprend la stratégie Naive de prédire la classe majoritaire à chaque fois, la loss est alors de -log(14638/len(df_train)) =1.4

In [7]:
df.loc[2].to_dict()

{'number': 2,
 'value': 0.8703358941441853,
 'datetime_start': Timestamp('2020-05-28 14:37:41.050529'),
 'datetime_complete': Timestamp('2020-05-28 15:27:06.984571'),
 'duration': Timedelta('0 days 00:49:25.934042'),
 'params_alpha': 0.2400936048140958,
 'params_booster': 'gbtree',
 'params_eta': 0.13219460009896047,
 'params_gamma': 0.0060702836128487215,
 'params_grow_policy': 'lossguide',
 'params_lambda': 0.3285133500747008,
 'params_max_depth': 9.0,
 'params_normalize_type': nan,
 'params_rate_drop': nan,
 'params_sample_type': nan,
 'params_skip_drop': nan,
 'state': 'COMPLETE'}

In [9]:
data,target = X_prep, y
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.1)
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(valid_x, label=valid_y)

xgb_params = {'alpha': 0.2400936048140958,
              'booster': 'gbtree',
              'eta': 0.13219460009896047,
              'gamma': 0.0060702836128487215,
              'grow_policy': 'lossguide',
              'lambda': 0.3285133500747008,
              'max_depth': 9,
              'objective': 'multi:softmax', 
              'eval_metric': 'mlogloss', 
              'seed': 23,
              'num_class':len(y.unique())
            }

bst = xgb.train(xgb_params, dtrain,20,evals=[(dvalid, "eval")])

[0]	eval-mlogloss:2.88022
[1]	eval-mlogloss:2.28407
[2]	eval-mlogloss:1.86922
[3]	eval-mlogloss:1.63167
[4]	eval-mlogloss:1.46906
[5]	eval-mlogloss:1.34692
[6]	eval-mlogloss:1.24930
[7]	eval-mlogloss:1.16259
[8]	eval-mlogloss:1.09278
[9]	eval-mlogloss:1.03138
[10]	eval-mlogloss:0.97851
[11]	eval-mlogloss:0.93084
[12]	eval-mlogloss:0.88758
[13]	eval-mlogloss:0.84909
[14]	eval-mlogloss:0.81378
[15]	eval-mlogloss:0.78253
[16]	eval-mlogloss:0.75379
[17]	eval-mlogloss:0.72781
[18]	eval-mlogloss:0.70488
[19]	eval-mlogloss:0.68442


In [20]:
y_pred = bst.predict( xgb.DMatrix(X_test_prep))

In [21]:
print('précision:',accuracy_score(y_pred,y_true))
print('présision pondéré: ', balanced_accuracy_score(y_pred,y_true))
print('f1_weighted : ',f1_score(y_pred,y_true,average='weighted'))

précision: 0.7315632167929381
présision pondéré:  0.6314742092997577
f1_weighted :  0.7547056234173378


In [24]:
import sklearn as sk
print(sk.metrics.classification_report(y_pred, y_true))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         9
         1.0       0.50      1.00      0.67         3
         2.0       0.00      0.00      0.00         3
         3.0       0.50      0.50      0.50         2
         4.0       0.50      0.50      0.50         2
         5.0       0.82      0.93      0.87        15
         6.0       0.33      1.00      0.50         1
         7.0       0.20      0.25      0.22         4
         8.0       0.33      0.67      0.44         3
         9.0       0.74      0.65      0.69        31
        10.0       0.77      0.67      0.71        15
        11.0       0.50      1.00      0.67         1
        12.0       0.00      0.00      0.00         0
        13.0       0.64      0.54      0.58        13
        14.0       0.22      1.00      0.36         2
        15.0       0.20      1.00      0.33         1
        16.0       0.86      0.74      0.80        43
        17.0       0.98    